# Palia Charts
Self-contained notebook with embedded chart cells.

Prereqs:
- Run your standard Cell 1 SQL flow first to produce `df_daily`.
- Then run this notebook top-to-bottom.


In [ ]:
"""
Lightweight shared style helpers for charts.

Safe to import from other cell scripts or `%run` first in a notebook.
"""

from __future__ import annotations

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


def ensure_theme() -> None:
    """Apply a clean, modern Seaborn theme suitable for notebooks and exports."""
    sns.set_theme(style="whitegrid", context="talk")


def percent_axis(ax: plt.Axes, axis: str = "y", decimals: int = 0) -> None:
    """Format the given axis as a percentage for values in [0,1]."""
    fmt = PercentFormatter(xmax=1.0, decimals=decimals)
    if axis.lower() == "y":
        ax.yaxis.set_major_formatter(fmt)
    else:
        ax.xaxis.set_major_formatter(fmt)



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Notebook Cell 2 — In-memory summary.

Use in a notebook by either:
- Pasting this whole file into Cell 2 and running it (recommended). It will
  resolve `df_daily` (or `_sqldf`/`sdf_daily`), compute `summary`, print
  shapes, and show a preview automatically.
- Or importing `process_summary_inline()` / `run_summary_cell()` and calling
  them yourself.
"""

import pandas as pd
import numpy as np
from typing import Any


def process_summary_inline(kpi_agg: pd.DataFrame) -> pd.DataFrame:
    df = kpi_agg.copy()

    # Coerce likely numeric columns
    numeric_cols = [
        "new_users","new_customers","daily_revenue","daily_transactions",
        "daily_active_users","daily_active_customers","daily_payers",
        "prior_day_active_users","d_over_d_returning_users",
        "prior_day_payers","d_over_d_returning_payers",
        "weekly_active_users","prior_week_active_users","w_over_w_returning_users",
        "weekly_active_customers","weekly_payers",
        "monthly_active_users","prior_month_active_users","m_over_m_returning_users",
        "monthly_active_customers","monthly_payers",
        "d7_regulars","d30_regulars","weekly_regulars","monthly_regulars",
        "prior_day_active_customers","d_over_d_returning_customers",
        "prior_week_active_customers","w_over_w_returning_customers",
        "prior_week_payers","w_over_w_returning_payers",
        "prior_month_active_customers","m_over_m_returning_customers",
        "prior_month_payers","m_over_m_returning_payers",
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)

    # Dates and labels
    df["metric_date_dt"] = pd.to_datetime(df["metric_date"], errors="coerce") if "metric_date" in df.columns else pd.to_datetime([])
    df = df.sort_values("metric_date_dt", ascending=True)
    df["day_of_week"] = df["metric_date_dt"].dt.strftime("%a")
    df["date"] = df["metric_date_dt"].dt.strftime("%m/%d")
    df["week_start"] = df["metric_date_dt"].dt.to_period("W").dt.start_time
    df["month_start"] = df["metric_date_dt"].dt.to_period("M").dt.start_time

    # Consistent aliases
    df["daily_new_users"] = df.get("new_users", 0)
    df["daily_new_customers"] = df.get("new_customers", 0)

    # Safe division
    def _as_series(x):
        # Avoid pandas ABC isinstance checks (can recurse in some environments)
        try:
            _ = x.index  # Series-like objects have .index
            return x
        except Exception:
            return pd.Series([x] * len(df), index=df.index)

    def safe_div(a, b):
        a = _as_series(a)
        b = _as_series(b)
        a_arr = pd.to_numeric(a, errors="coerce").to_numpy(dtype="float64", copy=False)
        b_arr = pd.to_numeric(b, errors="coerce").to_numpy(dtype="float64", copy=False)
        with np.errstate(divide='ignore', invalid='ignore'):
            res = np.divide(a_arr, b_arr, out=np.full_like(a_arr, np.nan, dtype="float64"), where=b_arr != 0)
        return pd.Series(res, index=df.index)

    # Simple derivations & rates
    df["dau_minus_new"] = df.get("daily_active_users", 0) - df.get("daily_new_users", 0)
    df["daily_customer_rate"] = safe_div(df.get("daily_active_customers", 0), df.get("daily_active_users", 0))
    df["daily_user_payer_rate"] = safe_div(df.get("daily_payers", 0), df.get("daily_active_users", 0))
    df["daily_customer_payer_rate"] = safe_div(df.get("daily_payers", 0), df.get("daily_active_customers", 0))

    # New users -> new customers conversions
    df["weekly_new_users"] = df.groupby("week_start")["daily_new_users"].transform("cumsum")
    df["monthly_new_users"] = df.groupby("month_start")["daily_new_users"].transform("cumsum")
    df["weekly_new_customers"] = df.groupby("week_start")["daily_new_customers"].transform("cumsum")
    df["monthly_new_customers"] = df.groupby("month_start")["daily_new_customers"].transform("cumsum")
    df["daily_new_user_to_customer_rate"] = safe_div(df["daily_new_customers"], df["daily_new_users"])
    df["weekly_new_user_to_customer_rate"] = safe_div(df["weekly_new_customers"], df["weekly_new_users"])
    df["monthly_new_user_to_customer_rate"] = safe_div(df["monthly_new_customers"], df["monthly_new_users"])

    # Revenue metrics
    df["daily_arpu"] = safe_div(df.get("daily_revenue", 0.0), df.get("daily_active_users", 0))
    df["daily_arppu"] = safe_div(df.get("daily_revenue", 0.0), df.get("daily_payers", 0))
    df["daily_arpc"] = safe_div(df.get("daily_revenue", 0.0), df.get("daily_active_customers", 0))
    if "daily_revenue" in df.columns:
        df["weekly_revenue"] = df.groupby("week_start")["daily_revenue"].transform("cumsum")
        df["monthly_revenue"] = df.groupby("month_start")["daily_revenue"].transform("cumsum")
        # Weekly/Monthly ARPU/ARPPU/ARPC
        if "weekly_active_users" in df.columns:
            df["weekly_arpu"] = safe_div(df["weekly_revenue"], df.get("weekly_active_users", 0))
        if "weekly_payers" in df.columns:
            df["weekly_arppu"] = safe_div(df["weekly_revenue"], df.get("weekly_payers", 0))
        if "weekly_active_customers" in df.columns:
            df["weekly_arpc"] = safe_div(df["weekly_revenue"], df.get("weekly_active_customers", 0))
        if "monthly_active_users" in df.columns:
            df["monthly_arpu"] = safe_div(df["monthly_revenue"], df.get("monthly_active_users", 0))
        if "monthly_payers" in df.columns:
            df["monthly_arppu"] = safe_div(df["monthly_revenue"], df.get("monthly_payers", 0))
        if "monthly_active_customers" in df.columns:
            df["monthly_arpc"] = safe_div(df["monthly_revenue"], df.get("monthly_active_customers", 0))

    # Retention rates
    df["dod_retention"] = safe_div(df.get("d_over_d_returning_users", 0), df.get("prior_day_active_users", 0))
    df["wow_retention"] = safe_div(df.get("w_over_w_returning_users", 0), df.get("prior_week_active_users", 0))
    df["mom_retention"] = safe_div(df.get("m_over_m_returning_users", 0), df.get("prior_month_active_users", 0))
    df["dod_payer_retention"] = safe_div(df.get("d_over_d_returning_payers", 0), df.get("prior_day_payers", 0))
    df["wow_payer_retention"] = safe_div(df.get("w_over_w_returning_payers", 0), df.get("prior_week_payers", 0))
    df["mom_payer_retention"] = safe_div(df.get("m_over_m_returning_payers", 0), df.get("prior_month_payers", 0))
    df["dod_customer_retention"] = safe_div(df.get("d_over_d_returning_customers", 0), df.get("prior_day_active_customers", 0))
    df["wow_customer_retention"] = safe_div(df.get("w_over_w_returning_customers", 0), df.get("prior_week_active_customers", 0))
    df["mom_customer_retention"] = safe_div(df.get("m_over_m_returning_customers", 0), df.get("prior_month_active_customers", 0))

    # Regular shares
    if "d7_regulars" in df.columns:
        df["d7_regular_share"] = safe_div(df["d7_regulars"], df.get("daily_active_users", 0))
    if "d30_regulars" in df.columns:
        df["d30_regular_share"] = safe_div(df["d30_regulars"], df.get("daily_active_users", 0))
    # DAU shares (stickiness)
    if "weekly_active_users" in df.columns:
        df["dau_wau_share"] = safe_div(df.get("daily_active_users", 0), df["weekly_active_users"])
    if "monthly_active_users" in df.columns:
        df["dau_mau_share"] = safe_div(df.get("daily_active_users", 0), df["monthly_active_users"])

    # Cohort conversion rates and ARPU/ARPC by horizon (if present from SQL)
    if "cohort_size" in df.columns:
        for h in (1, 7, 30, 60, 90):
            conv_col = f"cohort_d{h}_converted"
            rev_col = f"cohort_d{h}_revenue"
            if conv_col in df.columns:
                df[f"cohort_d{h}_conversion_rate"] = safe_div(df[conv_col], df["cohort_size"]) 
            if rev_col in df.columns:
                df[f"cohort_d{h}_arpu"] = safe_div(df[rev_col], df["cohort_size"]) 
                if conv_col in df.columns:
                    df[f"cohort_d{h}_arpc"] = safe_div(df[rev_col], df[conv_col]) 

    return df.sort_values("metric_date_dt", ascending=False).fillna(0)


def run_summary_cell(df_daily: Any) -> pd.DataFrame:
    """Ensure pandas input, compute summary, print shapes, and display a preview.

    - Accepts a pandas DataFrame or a Spark DataFrame (with toPandas()).
    - Prints df/summary shapes for quick sanity.
    - Displays a 5-row preview of the summary.
    - Returns the summary DataFrame.
    """
    # Spark → pandas if needed
    if hasattr(df_daily, "toPandas"):
        df_daily = df_daily.toPandas()

    if not isinstance(df_daily, pd.DataFrame):
        raise TypeError("df_daily must be a pandas DataFrame or Spark DataFrame with toPandas().")

    print(f"df_daily rows: {len(df_daily)} | cols: {len(df_daily.columns)}")
    summary = process_summary_inline(df_daily)
    print(f"summary rows: {len(summary)} | cols: {len(summary.columns)}")

    # Preview with robust fallbacks (some notebook envs have custom display behavior)
    try:
        try:
            from IPython.display import display as ipy_display  # type: ignore
            ipy_display(summary.head(5))
        except Exception:
            # Non-notebook or display unavailable
            print(summary.head(5).to_string(index=False))
    except RecursionError:
        # Rarely, some environments can trigger deep recursion in display; fallback to plain text
        try:
            print(summary.head(5).to_string(index=False))
        except Exception:
            print("Preview unavailable due to environment display recursion.")

    return summary


def _resolve_df_daily() -> pd.DataFrame:
    """Resolve df_daily from common notebook patterns: df_daily, _sqldf, sdf_daily."""
    g = globals()
    if 'df_daily' in g and not hasattr(g['df_daily'], 'toPandas'):
        # Assume plain pandas if it doesn't expose Spark's toPandas()
        return g['df_daily']
    if '_sqldf' in g:
        return g['_sqldf'].toPandas()
    if 'sdf_daily' in g:
        return g['sdf_daily'].toPandas()
    raise RuntimeError(
        "No input found for Cell 2. Define one in Cell 1:\n"
        "- df_daily (pandas)\n- _sqldf (Spark from %sql)\n- sdf_daily (Spark from Python)"
    )


if __name__ == "__main__":
    # Execute Cell 2 when this file is run in a notebook cell.
    df_daily = _resolve_df_daily()
    print("Cell 2 — df_daily shape:", df_daily.shape)
    summary = process_summary_inline(df_daily)
    print("Cell 2 — summary shape:", summary.shape)
    # Publish and preview
    globals()['summary'] = summary
    # Try to publish a Spark temp view for cross-language (R) access as `summary_kpis`
    try:
        from pyspark.sql import SparkSession  # type: ignore
        spark = SparkSession.builder.getOrCreate()
        spark.createDataFrame(summary).createOrReplaceTempView("summary_kpis")
        print("Cell 2 — published Spark temp view: summary_kpis")
    except Exception as _e:
        # Non-fatal if Spark not available
        pass
    try:
        display(summary.head(5))  # type: ignore[name-defined]
    except Exception:
        try:
            from IPython.display import display as _display  # type: ignore
            _display(summary.head(5))
        except Exception:
            print(summary.head(5).to_string(index=False))


In [ ]:
"""
Cell 5 — MAC × ARPMAC Run-rate Matrix

Purpose
- Compute a matrix of monthly MTX run-rate by combinations of MAC (Monthly Active Customers) and ARPMAC.
- Visualize as a heatmap to identify target corridors.

Data requirements
- monthly_mtx_revenue (float)
- mac (int) — monthly active customers
- Optionally, provide candidate grids for mac_values and arpmac_values.

Output
- Matplotlib/Seaborn heatmap figure.

Note: Wires to `summary` (Cell 2) if available to derive a grid around latest values.
"""

from __future__ import annotations

from typing import Iterable, Tuple

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


def ensure_theme() -> None:
    sns.set_theme(style="whitegrid", context="talk", palette="crest")


def build_matrix(
    mac_values: Iterable[int],
    arpmac_values: Iterable[float],
) -> pd.DataFrame:
    data = {"MAC": list(mac_values)}
    df = pd.DataFrame(data)
    for a in arpmac_values:
        df[f"ARPMAC={a:.2f}"] = df["MAC"] * a
    return df.set_index("MAC")


def to_annual(df_monthly: pd.DataFrame) -> pd.DataFrame:
    return df_monthly * 12.0


def plot_heatmap(
    annual_matrix: pd.DataFrame,
    title: str = "MTX Run Rate (Annualized) — MAC × ARPMAC",
    fmt: str = ".0f",
    cbar: bool = True,
):
    ensure_theme()
    fig, ax = plt.subplots(figsize=(12, 7))
    sns.heatmap(
        annual_matrix,
        annot=True,
        fmt=fmt,
        linewidths=0.5,
        linecolor="white",
        cbar=cbar,
        ax=ax,
    )
    ax.set_title(title)
    ax.set_xlabel("ARPMAC")
    ax.set_ylabel("MAC")
    plt.tight_layout()
    return fig, ax


def _derive_grid_from_summary(summary: pd.DataFrame) -> Tuple[list[int], list[float]]:
    s = summary.copy().sort_values("month_start")
    for c in ("monthly_active_customers", "monthly_revenue", "month_start"):
        if c not in s.columns:
            raise KeyError(f"summary missing {c}")
    last = s.dropna(subset=["monthly_active_customers", "monthly_revenue"]).tail(1)
    if last.empty:
        raise ValueError("summary has no monthly rows with required fields")
    mac0 = int(last["monthly_active_customers"].iloc[0])
    rev0 = float(last["monthly_revenue"].iloc[0])
    arpmac0 = rev0 / max(mac0, 1)
    mac_values = [max(1, int(mac0 * m)) for m in [0.8, 0.9, 1.0, 1.2, 1.4, 1.6]]
    arpmac_values = [max(0.01, round(arpmac0 * f, 2)) for f in [0.8, 0.9, 1.0, 1.2, 1.4, 1.6]]
    return sorted(set(mac_values)), sorted(set(arpmac_values))


def main() -> None:
    try:
        g = globals()
        if "summary" in g:
            mac_values, arpmac_values = _derive_grid_from_summary(g["summary"])
        else:
            raise KeyError("summary not found; using defaults")
    except Exception:
        mac_values = [1_000_000, 1_200_000, 1_500_000, 1_800_000, 2_160_000, 2_520_000, 2_880_000, 3_240_000, 3_600_000]
        arpmac_values = [3.00, 3.60, 4.20, 4.83, 5.43, 5.97]

    monthly_matrix = build_matrix(mac_values, arpmac_values)
    annual_matrix = to_annual(monthly_matrix)
    fig, _ = plot_heatmap(annual_matrix)
    try:
        plt.show()
    except Exception:
        fig.savefig("cell_5_mac_arpmac_matrix.png", dpi=160)


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 6 — Period Retention (DoD, WoW, MoM)

Scope
- Customer retention: prior active customers returning (DoD/WoW/MoM)
- Payer→Payer retention: prior payers returning as payers (DoD/WoW/MoM)

Notes
- Uses precomputed rates from Cell 2 `summary` to avoid SQL changes.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def plot_series(df: pd.DataFrame, x: str, y: str, title: str, outfile: str) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=df, x=x, y=y, marker="o")
    plt.title(title)
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig(outfile, dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"].copy().sort_values("metric_date_dt")
    s["date"] = pd.to_datetime(s["metric_date_dt"]).dt.date

    # Customers — DoD, WoW, MoM
    cust_cols = [
        ("dod_customer_retention", "Customer Retention — Day over Day", "cell_6_cust_dod_retention.png"),
        ("wow_customer_retention", "Customer Retention — Week over Week", "cell_6_cust_wow_retention.png"),
        ("mom_customer_retention", "Customer Retention — Month over Month", "cell_6_cust_mom_retention.png"),
    ]
    for col, title, outfile in cust_cols:
        if col in s.columns:
            plot_series(s, x="date", y=col, title=title, outfile=outfile)
        else:
            print("Skipping customers retention:", col, "not found.")

    # Payers — DoD, WoW, MoM (payer→payer)
    payer_cols = [
        ("dod_payer_retention", "Payer Retention — Day over Day", "cell_6_payer_dod_retention.png"),
        ("wow_payer_retention", "Payer Retention — Week over Week", "cell_6_payer_wow_retention.png"),
        ("mom_payer_retention", "Payer Retention — Month over Month", "cell_6_payer_mom_retention.png"),
    ]
    for col, title, outfile in payer_cols:
        if col in s.columns:
            plot_series(s, x="date", y=col, title=title, outfile=outfile)
        else:
            print("Skipping payer retention:", col, "not found.")


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 6 — Cohort Conversion by Horizon (D1/D7/D30/D60/D90)

Purpose
- Plot conversion curves by cohort using horizons available from Cell 1/2.
- Serves as a proxy until explicit cohort-based retention by month-since-first-MTX is available.

Data
- Uses `summary` from Cell 2. Requires columns: `cohort_size`, `cohort_d{h}_converted`.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def build_cohort_horizon_rates(summary: pd.DataFrame) -> pd.DataFrame:
    s = summary.copy()
    if "cohort_size" not in s.columns:
        raise KeyError("summary missing cohort_size; run Cell 1/2 with cohort rollup")
    s = s[s["cohort_size"] > 0].copy()
    s["cohort_month"] = pd.to_datetime(s["metric_date_dt"]).dt.to_period("M").dt.to_timestamp()
    horizons = [1, 7, 30, 60, 90]
    out = {"month": s["cohort_month"]}
    for h in horizons:
        conv_col = f"cohort_d{h}_converted"
        if conv_col in s.columns:
            rate = (s[conv_col] / s["cohort_size"]).replace([float("inf"), -float("inf")], float("nan"))
            out[f"d{h}_conversion"] = rate
    df = pd.DataFrame(out).drop_duplicates("month").sort_values("month")
    return df


def plot_horizon_rates(df: pd.DataFrame) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 7))
    value_cols = [c for c in df.columns if c != "month"]
    for col in value_cols:
        ax = sns.lineplot(data=df, x="month", y=col, marker="o", label=col)
    plt.title("Cohort Conversion by Horizon (D1/D7/D30/D60/D90)")
    plt.xlabel("Cohort Month")
    plt.ylabel("Conversion Rate")
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig("cell_6_cohort_conversion_horizons.png", dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"]
    try:
        df = build_cohort_horizon_rates(s)
        if len(df.columns) <= 1:
            print("No cohort horizon fields present; skipping chart.")
            return
        plot_horizon_rates(df)
    except Exception as e:
        print("Skipping cohort horizon chart:", e)


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 7 — Conversion Charts

Charts
- Period-based first-time conversion (monthly): new customers / new users in month
- Cohort D7 conversion: fraction of cohort converting by day 7
- Monthly conversion: unique payers / unique active players

Data
- Uses `summary` from Cell 2 if present. Falls back to placeholders otherwise.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def plot_time_series(df: pd.DataFrame, x: str, y: str, title: str, outfile: str) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=df, x=x, y=y, marker="o")
    plt.title(title)
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig(outfile, dpi=160)


def _monthly_period_conversion_from_summary(summary: pd.DataFrame) -> pd.DataFrame:
    s = summary.copy().sort_values("month_start")
    cols = ["month_start", "monthly_new_users", "monthly_new_customers"]
    for c in cols:
        if c not in s.columns:
            raise KeyError(f"summary missing {c}")
    m = s.groupby("month_start").agg(
        monthly_new_users=("monthly_new_users", "max"),
        monthly_new_customers=("monthly_new_customers", "max"),
    ).reset_index()
    m["period_first_time_conversion"] = (m["monthly_new_customers"] / m["monthly_new_users"]).replace([float("inf"), -float("inf")], float("nan"))
    return m[["month_start", "period_first_time_conversion"]].rename(columns={"month_start": "month"})


def _monthly_conversion_from_summary(summary: pd.DataFrame) -> pd.DataFrame:
    s = summary.copy().sort_values("month_start")
    cols = ["month_start", "monthly_payers", "monthly_active_users"]
    for c in cols:
        if c not in s.columns:
            raise KeyError(f"summary missing {c}")
    m = s.groupby("month_start").agg(
        monthly_payers=("monthly_payers", "max"),
        monthly_active_users=("monthly_active_users", "max"),
    ).reset_index()
    m["monthly_conversion"] = (m["monthly_payers"] / m["monthly_active_users"]).replace([float("inf"), -float("inf")], float("nan"))
    return m[["month_start", "monthly_conversion"]].rename(columns={"month_start": "month"})


def _cohort_d7_from_summary(summary: pd.DataFrame) -> pd.DataFrame:
    s = summary.copy()
    if "cohort_size" not in s.columns or "cohort_d7_converted" not in s.columns:
        raise KeyError("summary missing cohort fields")
    s = s[s["cohort_size"] > 0].copy()
    s["cohort_month"] = pd.to_datetime(s["metric_date_dt"]).dt.to_period("M").dt.to_timestamp()
    s["cohort_d7_conversion"] = (s["cohort_d7_converted"] / s["cohort_size"]).replace([float("inf"), -float("inf")], float("nan"))
    c = s.sort_values("cohort_month").drop_duplicates("cohort_month")
    return c[["cohort_month", "cohort_d7_conversion"]].rename(columns={"cohort_month": "month"})


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; charts will not render. Run Cell 2 first.")
        return
    s = g["summary"]

    try:
        df_period = _monthly_period_conversion_from_summary(s)
        plot_time_series(df_period, x="month", y="period_first_time_conversion", title="Period First-time Conversion (New Customers / New Users)", outfile="cell_7_period_first_time_conversion.png")
    except Exception as e:
        print("Skipping period first-time conversion:", e)

    try:
        df_monthly_conv = _monthly_conversion_from_summary(s)
        plot_time_series(df_monthly_conv, x="month", y="monthly_conversion", title="Monthly Conversion (Payers / Active Players)", outfile="cell_7_monthly_conversion.png")
    except Exception as e:
        print("Skipping monthly conversion:", e)

    try:
        df_c7 = _cohort_d7_from_summary(s)
        plot_time_series(df_c7, x="month", y="cohort_d7_conversion", title="Cohort D7 Conversion", outfile="cell_7_cohort_d7_conversion.png")
    except Exception as e:
        print("Skipping cohort D7 conversion:", e)


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 8 — MTX vs Entitlement Mix (Monthly and LTD)

Data
- Prefers `summary` from Cell 2. Looks for columns to compute shares:
  - Monthly: `monthly_total_revenue` and `monthly_mtx_revenue`; or `monthly_revenue` (MTX) plus entitlement fields if present.
  - LTD (optional): `ltd_total_revenue` and `ltd_mtx_revenue`.

Outputs
- Line charts for monthly share and LTD share (if fields are available).
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def compute_share(numer: pd.Series, denom: pd.Series) -> pd.Series:
    return (numer / denom).clip(lower=0, upper=1)


def plot_share(df: pd.DataFrame, x: str, y: str, title: str, outfile: str) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=df, x=x, y=y, marker="o")
    plt.title(title)
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig(outfile, dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"].copy().sort_values("month_start")

    total_col, mtx_col = None, None
    if {"monthly_total_revenue", "monthly_mtx_revenue"}.issubset(s.columns):
        total_col, mtx_col = "monthly_total_revenue", "monthly_mtx_revenue"
    elif "monthly_revenue" in s.columns and "monthly_entitlement_revenue" in s.columns:
        s["monthly_total_revenue"] = s["monthly_revenue"] + s["monthly_entitlement_revenue"]
        total_col, mtx_col = "monthly_total_revenue", "monthly_revenue"
    else:
        print("Skipping monthly mix: total revenue fields not found.")

    if total_col and mtx_col:
        m = s.groupby("month_start").agg(
            total=(total_col, "max"),
            mtx=(mtx_col, "max"),
        ).reset_index()
        m["mtx_share"] = compute_share(m["mtx"], m["total"]) 
        plot_share(m.rename(columns={"month_start": "month"}), x="month", y="mtx_share", title="Monthly MTX Share of Total Revenue", outfile="cell_8_monthly_mtx_share.png")

    if {"ltd_total_revenue", "ltd_mtx_revenue"}.issubset(s.columns):
        l = s.sort_values("metric_date_dt").tail(1)[["ltd_total_revenue", "ltd_mtx_revenue"]]
        share = compute_share(l["ltd_mtx_revenue"], l["ltd_total_revenue"]).iloc[0]
        print(f"LTD MTX share: {share:.2%}")


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 9 — LTV Charts

Charts (if data is available)
- Entitlement LTV = ltd_entitlement_revenue / ltd_unique_new_players
- MTX Payer LTV = ltd_mtx_revenue / ltd_unique_mtx_purchasers
- Player MTX LTV = ltd_mtx_revenue / ltd_unique_players

Note: If LTD fields are not available in `summary`, this cell will skip.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme


def plot_ltvs(df: pd.DataFrame) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    for col in [c for c in df.columns if c != "month"]:
        sns.lineplot(data=df, x="month", y=col, marker="o", label=col)
    plt.title("LTV Metrics Over Time")
    plt.legend()
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig("cell_9_ltv_charts.png", dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"].copy().sort_values("month_start")

    needed_any = [
        {"ltd_mtx_revenue", "ltd_unique_mtx_purchasers"},
        {"ltd_mtx_revenue", "ltd_unique_players"},
        {"ltd_entitlement_revenue", "ltd_unique_new_players"},
    ]
    if not any(cols.issubset(s.columns) for cols in needed_any):
        print("Skipping LTV charts: LTD fields not found in summary.")
        return

    df = pd.DataFrame({"month": s["month_start"].dropna().unique()})
    df = df.sort_values("month")

    if {"ltd_mtx_revenue", "ltd_unique_mtx_purchasers"}.issubset(s.columns):
        last = s.dropna(subset=["ltd_mtx_revenue", "ltd_unique_mtx_purchasers"]).drop_duplicates("month_start", keep="last")
        df = df.merge(
            last[["month_start", "ltd_mtx_revenue", "ltd_unique_mtx_purchasers"]].rename(columns={"month_start": "month"}),
            on="month", how="left"
        )
        df["ltv_mtx_payer"] = (df["ltd_mtx_revenue"] / df["ltd_unique_mtx_purchasers"]) 

    if {"ltd_mtx_revenue", "ltd_unique_players"}.issubset(s.columns):
        last = s.dropna(subset=["ltd_mtx_revenue", "ltd_unique_players"]).drop_duplicates("month_start", keep="last")
        df = df.merge(
            last[["month_start", "ltd_mtx_revenue", "ltd_unique_players"]].rename(columns={"month_start": "month", "ltd_mtx_revenue": "ltd_mtx_revenue_players"}),
            on="month", how="left"
        )
        df["ltv_player_mtx"] = (df["ltd_mtx_revenue_players"] / df["ltd_unique_players"]) 

    if {"ltd_entitlement_revenue", "ltd_unique_new_players"}.issubset(s.columns):
        last = s.dropna(subset=["ltd_entitlement_revenue", "ltd_unique_new_players"]).drop_duplicates("month_start", keep="last")
        df = df.merge(
            last[["month_start", "ltd_entitlement_revenue", "ltd_unique_new_players"]].rename(columns={"month_start": "month"}),
            on="month", how="left"
        )
        df["ltv_entitlement"] = (df["ltd_entitlement_revenue"] / df["ltd_unique_new_players"]) 

    value_cols = [c for c in df.columns if c != "month"]
    keep = [c for c in value_cols if df[c].notna().any()]
    if not keep:
        print("No LTV series with data; skipping chart.")
        return
    plot_ltvs(df[["month"] + keep])


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 10 — Activity Mix: Customers as Share of Active Players

Chart
- Customers / Active Players (monthly), using `summary` from Cell 2.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def plot_customers_share(df: pd.DataFrame) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=df, x="month", y="customers_share", marker="o")
    plt.title("Customers Share of Active Players")
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig("cell_10_activity_mix_customers_share.png", dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"].copy()
    cols = ["month_start", "monthly_active_customers", "monthly_active_users"]
    for c in cols:
        if c not in s.columns:
            print("Skipping activity mix: missing", c)
            return
    m = s.groupby("month_start").agg(
        mac=("monthly_active_customers", "max"),
        mau=("monthly_active_users", "max"),
    ).reset_index()
    m = m.rename(columns={"month_start": "month"})
    m["customers_share"] = (m["mac"] / m["mau"]).clip(lower=0, upper=1)
    plot_customers_share(m[["month", "customers_share"]])


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 11 — VC Price Realization vs. List

Effective VC price = realized_mtx_vc_revenue / total_vc_units_consumed_at_list_price
Optionally adjust numerator by excluding discounts or including net-of-fees.
Currently optional — may be unused if VC pricing is out of scope.
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme


def plot_price_realization(df: pd.DataFrame) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=df, x="month", y="effective_price", marker="o", label="Effective")
    if "list_price" in df.columns:
        sns.lineplot(data=df, x="month", y="list_price", marker="o", label="List")
    plt.title("VC Price Realization vs. List")
    plt.legend()
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig("cell_11_vc_price_realization.png", dpi=160)


def main() -> None:
    # Placeholder; requires VC ledger mapping. Skipping by default.
    pass


if __name__ == "__main__":
    main()



In [ ]:
"""
Cell 12 — Dormancy Rate

Share of customers purchasing in the month; complement suggests dormancy.
Metric: monthly_payers / monthly_active_customers (from `summary`).
"""

from __future__ import annotations

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from cell_style import ensure_theme, percent_axis


def plot_dormancy(df: pd.DataFrame) -> None:
    ensure_theme()
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(data=df, x="month", y="purchase_rate", marker="o")
    plt.title("Customers Purchasing in Month (Dormancy Complement)")
    percent_axis(ax)
    plt.tight_layout()
    try:
        plt.show()
    except Exception:
        plt.savefig("cell_12_dormancy_rate.png", dpi=160)


def main() -> None:
    g = globals()
    if "summary" not in g:
        print("summary not found; run Cell 2 first.")
        return
    s = g["summary"].copy()
    cols = ["month_start", "monthly_payers", "monthly_active_customers"]
    for c in cols:
        if c not in s.columns:
            print("Skipping dormancy: missing", c)
            return
    m = s.groupby("month_start").agg(
        monthly_payers=("monthly_payers", "max"),
        mac=("monthly_active_customers", "max"),
    ).reset_index().rename(columns={"month_start": "month"})
    m["purchase_rate"] = (m["monthly_payers"] / m["mac"]).clip(lower=0, upper=1)
    plot_dormancy(m[["month", "purchase_rate"]])


if __name__ == "__main__":
    main()

